In [1]:
import pandas as pd 
import win32com.client as win32
import re
from datetime import *
import calendar
import os

In [2]:
outlook = win32.Dispatch('Outlook.Application').GetNamespace("MAPI")
root_folder = outlook.Folders['Karun.Kalsi@plains.com']
inbox = root_folder.Folders["Inbox"]
path = r"H:\LPG\Analyst Pool\Karun\github\localPython\ice_eod\test_outputs"
test_env_inbox = inbox.Folders['Test Enviroment']
messages = test_env_inbox.Items
# today =date.today()
today = date(2023,12,12)
test_day = today.day
test_month = today.month
test_year = today.year


In [3]:
year = date.today().year
month_num = date.today().month
month_name = calendar.month_name[month_num]
month_abb = calendar.month_abbr[month_num]
day = date.today().day

new_path = os.path.join(path,str(year))

if not os.path.isdir(new_path):
    os.mkdir(new_path)
new_path = os.path.join(new_path,f"{month_num}. {month_name} {year}")
if not os.path.isdir(new_path):
    os.mkdir(new_path)
new_path = os.path.join(new_path,str(day))    
if not os.path.isdir(new_path):
    os.mkdir(new_path)    

In [4]:
def saveattachemnts(subject):
    for message in messages:
        if message.Subject == subject and message.Senton.date() == today:
            attachments = message.Attachments
            attachment = attachments.Item(1)
            for attachment in message.Attachments:
                attachment.SaveAsFile(f"{path}/{str(attachment)}")
                if message.Subject == subject and message.Unread:
                    message.Unread = False
                break

In [5]:
subjects = ["ICE Data: Cleared Canadian Oil Settlement",
"ICE Data: NGX Gas Settlement",
"ICE Data: NGX Power Settlement",
"ICE Data: Cleared Oil Settlement",
"ICE Data: Cleared NGL Settlement",
"ICE Data: Cleared Gas Settlement"]
for subject in subjects:
    saveattachemnts(subject)

In [36]:
def export_to_excel(df, name):
    export_loc = os.path.join(new_path, f'{name}_{month_abb}_{day}_{year}.xlsx')
    writer = pd.ExcelWriter(export_loc, engine="xlsxwriter")
    df.to_excel(writer, sheet_name = "Sheet1", index = False)
    
    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    
    format1 = workbook.add_format({'bg_color': "#C4D79B", 
                                   'font_name':"calibri"})
    format_headers = workbook.add_format({'bold': False, 
                                          'font_name':"calibri",})
    for row_num, value in enumerate(df['Value']):
        worksheet.write(row_num+1, 4, value, format1)
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, format_headers) 
    worksheet.autofit()
    writer.close()
   

In [46]:
def merge_data(import_file, name, filter_to):
    todays_data = pd.read_excel(os.path.join(path, f"{import_file}{test_year}_{test_month}_{test_day}.xlsx"))
    template = pd.read_excel(os.path.join(template_path, f'{name}_Template.xlsx'))
    cols = template.columns
    template[["QuoteFromDate","QuoteToDate"]] = today
    template['DeliveryPeriod'] = pd.to_datetime(template['DeliveryPeriod']).dt.date
    todays_data = todays_data.loc[todays_data['CONTRACT'].isin(filter_to)]
    todays_data['STRIP'] = pd.to_datetime(todays_data['STRIP']).dt.date 
    template = 
    template = template.join(todays_data[['STRIP','SETTLEMENT PRICE']].set_index('STRIP'), on = ['DeliveryPeriod','PriceCurveName'])
    template = template.drop(columns = "Value")
    template = template.rename(columns={'SETTLEMENT PRICE':"Value"})
    template = template[cols]
    return template

In [47]:
template_path = os.path.join(path,'templates')
for subject in subjects:
    if subject == "ICE Data: Cleared Canadian Oil Settlement":
        name = "AESO 7x24-FWD"
        template = merge_data("ngxcleared_power_",name, ['XCU'])
        export_to_excel(template, name)
    if subject == "ICE Data: Cleared Oil Settlement":
        name = "ICE_CRUDE"
        template = merge_data('icecleared_oil_', name, ['ARV', 'NGE','NGL'])
        # export_to_excel(template,name)
print(len(template))

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [ ]:
template

,PriceCurveName,DeliveryPeriod,QuoteFromDate,QuoteToDate,Value,EstimateorActual,PriceType
0,ICE Crude Diff - ARV Argus WCS Houston,2024-01-01,2023-12-12,2023-12-12,-7.91200,Actual,Settle
0,ICE Crude Diff - ARV Argus WCS Houston,2024-01-01,2023-12-12,2023-12-12,1.36625,Actual,Settle
0,ICE Crude Diff - ARV Argus WCS Houston,2024-01-01,2023-12-12,2023-12-12,1.38000,Actual,Settle
1,ICE Crude Diff - ARV Argus WCS Houston,2024-02-01,2023-12-12,2023-12-12,-8.35000,Actual,Settle
1,ICE Crude Diff - ARV Argus WCS Houston,2024-02-01,2023-12-12,2023-12-12,1.37125,Actual,Settle
...,...,...,...,...,...,...,...
60,ICE Mt. Belvieu TET Natural Gasoline (OPIS) Fu...,2024-11-01,2023-12-12,2023-12-12,1.33500,Actual,Settle
60,ICE Mt. Belvieu TET Natural Gasoline (OPIS) Fu...,2024-11-01,2023-12-12,2023-12-12,1.34875,Actual,Settle
61,ICE Mt. Belvieu TET Natural Gasoline (OPIS) Fu...,2024-12-01,2023-12-12,2023-12-12,-7.75000,Actual,Settle
61,ICE Mt. Belvieu TET Natural Gasoline (OPIS) Fu...,2024-12-01,2023-12-12,2023-12-12,1.33125,Actual,Settle
